In [ ]:
!pip install pymysql

In [ ]:
import mod_sql as ms
import pandas as pd

In [70]:
## 대면
sql_class = ms.Database("172.16.12.149", 'ubion', "1234")
## 비대면면
sql_class_2 = ms.Database("darkpreist.iptime.org", "ubion", "1234")

1. emp라는 변수에 sql에 있는 emp table의 전체 데이터를 데이터프레임으로 저장
2. dept라는 변수에 sql에 있는 dept table의 전체 데이터를 데이터프레임으로 저장

In [ ]:
sql = """
    select 
    * 
    from 
    emp
"""
emp = sql_class.executeAll(sql)
emp.head()

In [ ]:
sql = """
    select 
    * 
    from 
    dept
"""
dept = sql_class.executeAll(sql)
dept

## pandas 복습
1. 새로운 파생변수 생성 job_code
    - salesman은 1, manager는 2, 그 외의 직업은 3
    - 컬럼은 EMPNO, ENAME, job_code만 출력

In [ ]:
emp['JOB']

## apply, def 
def change(x):
    if x == 'SALESMAN':
        return 1
    elif x == "MANAGER":
        return 2
    else :
        return 3
emp['JOB'].apply(change)

## apply, lambda

emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else 
    (2 if (x == 'MANAGER') else 3))

emp2 = emp.copy()

## for문
for i in emp2.index:
    if emp2.loc[i, 'JOB'] == 'SALESMAN':
        emp2.loc[i, 'job_code'] = 1
    elif emp2.loc[i, 'JOB'] == 'MANAGER':
        emp2.loc[i, 'job_code'] = 2
    else :
        emp2.loc[i, 'job_code'] = 3
emp2[["EMPNO", "ENAME", "job_code"]]


In [ ]:
## 위의 결과를 sql 쿼리문을 이용하여 생성
sql = """
    select 
    EMPNO, 
    ENAME, 
    JOB, 
    CASE 
        when JOB = 'SALESMAN'
        then 1 
        when JOB = 'MANAGER' 
        then 2
        else 3
    END as job_code 
    from 
    EMP
"""
sql_class.executeAll(sql)

### pandas 복습 문제
1. emp에서 SAL 컬럼의 값이 2000 이상이고 5000 미만인 사원 정보를 출력

In [ ]:
emp[(emp['SAL'] >= 2000) & (emp['SAL'] < 5000)]

In [ ]:
sql = """
    select 
    * 
    from 
    emp 
    where 
    SAL >= 2000 
    and 
    SAL < 5000
"""
sql_class.executeAll(sql)

In [ ]:
sql = """
    select
    * 
    from 
    emp 
    where 
    SAL 
    between 
    2000 and 4999
"""
sql_class.executeAll(sql)

### pandas 복습
1. JOB 컬럼의 데이터를 변환 
2. 'SALESMAN' 값을 'SALES' 변환

In [ ]:
# case1
emp['JOB'].replace('SALESMAN', 'SALES')

# case2 
emp['JOB'].apply(lambda x : "SALES" if (x == 'SALESMAN') else x)

In [ ]:
## sql 쿼리문 버전
sql = """
    select 
    JOB, 
    replace(JOB, 'SALESMAN', 'SALES') as JOB_ 
    from 
    emp
"""
sql_class.executeAll(sql)

### pandas
1. 2개의 컬럼의 데이터를 하나의 컬럼에 삽입
2. emp 데이터에서 ENAME, JOB 두 데이터를 결합
3. ENAME_ 컬럼에 ENAME(JOB) 형태로 데이터를 생성 

In [ ]:
emp2 = emp.copy()
emp2['ENAME_'] = emp2["ENAME"] + '(' + emp2["JOB"] + ')'
emp2['ENAME_']

# ENAME(JOB)-> ENAME+ (+ JOB+ )

In [ ]:
## sql 쿼리문 
sql = """
    select 
    concat(ENAME, '(' ,JOB, ')' ) as ENAME_
    from 
    emp
"""
sql_class.executeAll(sql)

### pandas 
1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000 이상이면 High, 2000 미만이면 Low

In [ ]:
# case1
emp['SAL'].apply(lambda x : 'High' if (x >= 2000) else 'Low')

# case2 
def change(x):
    if x >= 2000:
        return 'High'
    else : 
        return "Low"
emp['SAL'].apply(change)

# case3
emp2 = emp.copy()

for i in emp2.index:
    if emp2.loc[i, 'SAL'] >= 2000:
        emp2.loc[i, 'H/L'] = 'High'
    else :
        emp2.loc[i, 'H/L'] = 'Low'
emp2['H/L']

In [ ]:
## sql 쿼리문
sql = """
    select 
    SAL, 
    if(SAL >= 2000, 'High', 'Low') as `H/L` 
    from 
    emp
"""
sql_class.executeAll(sql)

In [ ]:
sql = """
    select 
    SAL, 
    case 
        when SAL >= 2000
        then 'High'
        else 'Low'
    end as `H/L`
    from 
    emp
"""
sql_class.executeAll(sql)

In [ ]:
sql = """
    select 
    SAL, 
    if(SAL >=2000, 'High', if(SAL >= 1000, 'Mid', 'Low')) as `HML`
    from emp
"""
sql_class.executeAll(sql)

### pandas 
1. emp와 dept 데이터프레임을 결합 
2. 조건은 emp에 있는 deptno, dept에 있는 deptno 가 같은 경우 
3. 데이터프레임을 조인 결합

In [ ]:
## 조인결합 -> merge()
pd.merge(emp, dept, on='DEPTNO', how='left')

In [ ]:
sql = """
    select 
    * 
    from 
    emp 
    left join 
    dept 
    on 
    emp.DEPTNO = dept.DEPTNO
"""
sql_class.executeAll(sql)

### 이중쿼리문
1. 부서의 위치가 'NEW YORK', 'DALLAS' 가 아닌 소속 사원의 정보 출력
    - dept 테이블에서 위치가 'NEW YORK', 'DALLAS' 아닌 부서 번호 출력
    - emp table에서 위의 결과인 소속 사원 정보 출력

In [66]:
sql = """
    select 
    DEPTNO 
    from 
    dept 
    where 
    LOC not in ('NEW YORK', 'DALLAS')
"""
sql2 = """
    select 
    DEPTNO 
    from 
    dept 
    where 
    LOC != 'NEW YORK' 
    and 
    LOC != 'DALLAS'
"""
sql_class.executeAll(sql2)

,DEPTNO
0,30.0
1,40.0


In [68]:
sql = """
    select 
    * 
    from 
    emp 
    where 
    DEPTNO = 30 
    or 
    DEPTNO = 40
"""
sql2 = """
    select 
    * 
    from 
    emp 
    where 
    DEPTNO in (30, 40)
"""
sql_class.executeAll(sql2)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [69]:
sql = """
    select * from emp 
    where DEPTNO in 
        (select DEPTNO from dept 
        where 
        LOC not in ('NEW YORK', 'DALLAS')
        )
"""
sql_class.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


### pandas 
1. tran_1, tran_2, tran_d_1, tran_d_2 데이터베이스에서 select문을 이용해서 데이터프레임으로 저장
2. tran_1, tran_2 단순한 행 결합(유니언 결합)
3. tran_d_1, tran_d_2 단순한 행 결합 (유니언결합)
4. tran, tran_d를 특정 조건에 맞게 조인 결합

In [71]:
sql = """
    select 
    * 
    from 
    tran_1
"""
tran_1 = sql_class.executeAll(sql)

In [72]:
sql = """
    select 
    * 
    from 
    tran_2
"""
tran_2 = sql_class.executeAll(sql)

In [73]:
sql = """
    select 
    * 
    from 
    tran_d_1
"""
tran_d_1 = sql_class.executeAll(sql)

In [74]:
sql = """
    select 
    * 
    from 
    tran_d_2
"""
tran_d_2 = sql_class.executeAll(sql)

In [75]:
tran = pd.concat([tran_1, tran_2])

In [76]:
tran_d = pd.concat([tran_d_1, tran_d_2])

In [77]:
tran.head(1)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502


In [78]:
tran_d.head(1)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1


In [79]:
transaction = pd.merge(tran, tran_d, on='transaction_id', how='left')

In [80]:
transaction.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1


In [ ]:
## sql 쿼리문
sql = """
    select * from tran_1 
    union 
    select * from tran_2
"""
sql_class.executeAll(sql)

In [ ]:
sql = """
    select * from 
    (select * from tran_1 
    union 
    select * from tran_2) as tran 
    left join 
    (select * from tran_d_1 
    union 
    select * from tran_d_2) as tran_d 
    on 
    tran.transaction_id = tran_d.transaction_id
"""
sql_class.executeAll(sql)

In [ ]:
sql = """
    select 
    * 
    from 
    tran_1 
    left join 
    tran_d_1
    on 
    tran_1.transaction_id = tran_d_1.transaction_id
"""